In [1]:
from pathlib import Path
import pandas as pd

df = pd.read_pickle(Path.cwd() / "pickles" /"parsed.pickle")

Hello

In [ ]:
## Get column names
df.columns
# initialise dictionary
adj_dict = {}
# Loops thru each row in the df and gets the spacy doc
for index, row in df.iterrows():
    print("-"*30)
    print(row['title'])
    doc = row['parsed']
    # Loop thru each word of each spacy doc
    for token in doc:
        if token.pos_ == "ADJ":
            adj = token.lemma_
            adj_dict[adj] = adj_dict.get(adj, 0) + 1

# Convert dictionary to lsit of tuples
adj_list = list(adj_dict.items())


print(adj_list)

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Create fake data to test function
data = {'title': ['novel1', 'novel2', 'novel3'],
        'text': ['there was a pink happy cat',
                 'the broccoli was soft green and hot',
                 'the athlete went running']}

df = pd.DataFrame(df)
# Add parsed column 
parsed_list = []
for index, row in df.iterrows():
    parsed_obj = nlp(row['text'])
    parsed_list.append(parsed_obj)
    
df['parsed'] = parsed_list

In [7]:
# Test subject_by_verb_count
import importlib
import PartOne as po

importlib.reload(po)


df_mini = df.iloc[[0]]
#print(df_mini)
out = po.subjects_by_verb_count(df, 'to hear')

for title, pair in out.items():
    print(f"{title}:")
    if pair:
        for pair_dict in pair:
            print(f"\t{pair_dict}")


[nltk_data] Downloading package cmudict to
[nltk_data]     /Users/enmanuelmoreno/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


Sense_and_Sensibility:
	{('hear', 'I'): 32}
	{('hear', 'you'): 19}
	{('hear', 'she'): 14}
	{('hear', 'they'): 6}
	{('hear', 'Elinor'): 6}
	{('hear', 'he'): 6}
	{('hear', 'Jennings'): 3}
	{('hear', 'we'): 2}
	{('hear', 'Brandon'): 1}
	{('hear', 'both'): 1}
North_and_South:
	{('hear', 'she'): 59}
	{('hear', 'I'): 47}
	{('hear', 'he'): 23}
	{('hear', 'you'): 15}
	{('hear', 'they'): 13}
	{('hear', 'Margaret'): 10}
	{('hear', 'we'): 5}
	{('hear', 'Thornton'): 3}
	{('hear', 'who'): 3}
	{('hear', 'yo'): 2}
A_Tale_of_Two_Cities:
	{('hear', 'I'): 23}
	{('hear', 'he'): 19}
	{('hear', 'you'): 12}
	{('hear', 'she'): 11}
	{('hear', 'they'): 5}
	{('hear', 'Monseigneur'): 2}
	{('hear', 'one'): 1}
	{('hear', 'Jerry'): 1}
	{('hear', 'D’ye'): 1}
	{('hear', 'we'): 1}
Erewhon:
	{('hear', 'I'): 38}
	{('hear', 'he'): 4}
	{('hear', 'they'): 3}
	{('hear', 'she'): 2}
	{('hear', 'we'): 1}
	{('hear', 'who'): 1}
	{('hear', 'destruction'): 1}
	{('hear', 'machine'): 1}
	{('hear', 'one'): 1}
The_American:
	{('hear',